In [1]:
#preprocessing survey data downloaded from infoflora

import pandas as pd
import geopandas as gp

In [2]:
# Import the updated set of data files from infoflora 
first = pd.read_csv("C:\\Users\\visitor\\Documents\\GitHub\\flora-biel-2022\\data\\surveys\\obs_export_2022-05-15_13h34.csv",encoding = "utf-16",sep = "\t")
second = pd.read_csv("C:\\Users\\visitor\\Documents\\GitHub\\flora-biel-2022\\data\\surveys\\obs_export_2022-05-15_13h35.csv",encoding = "utf-16",sep = "\t")
all_data = pd.concat([first,second],ignore_index = True)

In [3]:
# make a dictionary of the columns with their definition

info_cols = {'obs_id':'unique identifier','date':'date of survey','x':'coordinate gis', 'y':'coordinate gis', 'taxon.taxon_id':'unique species identifier for infoflora reference',
            'taxon.no_isfs': 'unique species identifier for infoflora reference','taxon_orig':'original species name selected','municipality.name' : 'name of municipality sample taken',
            'phenology' : 'stage of growth','locality_descript' : 'name of survey site','specimen_type' : 'if there is a photo','v_co_canton' : 'name of canton sample taken',
            'xy_precision' : 'accuracy of x,y coordinates','altitude_min' : 'altitude for observations','cover':'coverage of area with the species'}
# make a new df out of those columns
ad = all_data[info_cols.keys()].copy()

# set date to datetime format
ad['md'] = pd.to_datetime(ad.date)
ad['slug'] = ad.taxon_orig.str.lower().str.replace("cf\.","").str.replace("l\.","").str.replace("aggr\.","").str.replace("  ","").str.replace(" ","")
ad['locality_descript'] = ad.locality_descript.str.lower().str.replace(" ","")

def to_slug(x):
    data = x.split()
    try:
        #checks if sp. or cf. in second position ( eg rosa sp. or Achillea cf. millefolium)
        if len(data[1])==3:
            # if cf. joins element 0 and 2
            try:
                mydata = "".join([data[0],data[2]]).lower()
            # if species returns only the genus
            except:
                mydata=data[0].lower()
        # else returns the first two 
        else:
            mydata = "".join(data[0:2]).lower()
        return mydata
    # returns the family lowercase (e.g. poaceae)
    except:
        return data[0].lower()

ad["slug"] = ad.taxon_orig.apply(lambda x: to_slug(x))   
    
# all poaceae ids
dp = ad[ad['taxon_orig']=='Poaceae']

# all plants ided in 2022
d22 = ad[ad.md>'01-01-2022']

C:\Users\visitor\AppData\Local\Temp\ipykernel_10048\968194841.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  ad['slug'] = ad.taxon_orig.str.lower().str.replace("cf\.","").str.replace("l\.","").str.replace("aggr\.","").str.replace("  ","").str.replace(" ","")


In [4]:
# save to csv
ad.to_csv("C:\\Users\\visitor\\Documents\\GitHub\\flora-biel-2022\\output\\alldata.csv")
dp.to_csv("C:\\Users\\visitor\\Documents\\GitHub\\flora-biel-2022\\output\\poaceae.csv")
d22.to_csv("C:\\Users\\visitor\\Documents\\GitHub\\flora-biel-2022\\output\\data22.csv")